# Пункт 1

In [26]:
import numpy as np
import pandas as pd


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        le_x_prior = self.lexicon * self.prior
        return rownorm(le_x_prior) #Ваш код тут

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        speak = self.speaker().T
        speek_x_prior = speak * self.prior
        return rownorm(speek_x_prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['шляпа', 'очки', 'усы']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[float(1/3), float(1/3), float(1/3)], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())


Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0.0
очки,0.000000,0.0,1.0,0.0
усы,0.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
шляпа,0.5,0.5,0.0
очки,0.0,0.0,1.0
усы,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
шляпа,0.75,0.25,0.0
очки,0.00,0.00,1.0
усы,0.00,1.00,0.0


# Пункт 2

**1) Можно ли вывести скалярную импликатуру из таблицы прагматического слушающего? обоснуйте свой ответ**

По данным получается, что менее информативной является шляпа (потому что шляпа выбирает множество из 2х обьектов), а более информативными являются усы (выбирает множество из одного обьекта. То есть у нас есть шкала шляпа-усы. Использование менее информативного элемента сигнализирует об отрицании более информативного. Тогда можно вывести импликатуру, что шляпа не усы.

**2) что изменится, с точки зрения вычисления импликатуры, если поменять исходные вероятности на P(r1) = 0.1, P(r2) = P(r3) = 0.45? Приведите соответствующую таблицу распределения вероятностей для прагматического слушающего и прокомментируйте её. При расчетах, можно использовать код из пункта 1.**

In [4]:
msgs = ['шляпа', 'очки', 'усы']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs, columns=states)

print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0.0
очки,0.0,0.0,1.0,0.0
усы,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
шляпа,0.181818,0.818182,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


При изменении априорных вероятностей теперь труднее сказать о наличии импликатуры, потому что даже если слушающий поймет, что шляпа не усы, исходная вероятность скорее выбрать обьект r2 может пересилить.

Теперь прагматический слушатель больше будет сомневаться, так как при употреблении шляпы вероятность указать на второе состояние выросла и стала ближе к вероятности указать на то же второе состояние, сказав усы.

# Пункт 3

## игра 1 

In [6]:
msgs = ['шляпа', 'очки', 'усы']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0],
    [1.0, 0.0, 1.0]], index=msgs, columns=states)

print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod = RSA(lexicon=lex, prior=[float(1/3), float(1/3), float(1/3)], costs=[0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.000000,1.0,0.0,0.0
очки,1.000000,1.0,0.0,0.0
усы,1.000000,0.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
шляпа,0.0,1.0,0.0
очки,0.5,0.5,0.0
усы,0.5,0.0,0.5



Pragmatic speaker


,шляпа,очки,усы
r1,0.000000,0.500000,0.5
r2,0.666667,0.333333,0.0
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
шляпа,0.000000,1.0,0.000000
очки,0.600000,0.4,0.000000
усы,0.333333,0.0,0.666667


В игре 1 мы получили 2 скалярные импликатуры:
1) очки – не шляпа
2) усы – не очки

In [43]:
alph = [0.0, 0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 4.0, 10.0]

pri = [[0.3, 0.4, 0.3],
       [0.2, 0.6, 0.2],
       [0.1, 0.8, 0.1]]

cost = [[0, -7, 0],
        [0, -6, 0],
        [0, -5, 0],
        [0, -4, 0],
        [0, -3, 0],
        [0, -2, 0],
        [0, -1, 0]]

In [21]:
def display_rsa(p=[float(1/3), float(1/3), float(1/3)], c=[0.0, 0.0, 0.0], a=1.0):
    msgs = ['шляпа', 'очки', 'усы']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [0.0, 1.0, 0.0],
        [1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=p, costs=c, alpha=a)

    display_reference_game(basic_mod)

#     print("\nLiteral listener")
#     display(basic_mod.literal_listener())

#     print("\nPragmatic speaker")
#     display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())    
    

In [34]:
for al in alph:
    display_rsa(a=al)

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.000000,1.0,0.0,0.0
очки,1.000000,1.0,0.0,0.0
усы,1.000000,0.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,0.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,NaN,1.0,NaN
очки,0.500000,0.5,NaN
усы,0.333333,NaN,0.666667


Even priors and all-0 message costs



...

,r1,r2,r3
шляпа,0.000000,1.000000,0.000000
очки,0.894737,0.105263,0.000000
усы,0.333333,0.000000,0.666667


Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.000000,1.0,0.0,0.0
очки,1.000000,1.0,0.0,0.0
усы,1.000000,0.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,10.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.000000,1.000000,0.000000
очки,0.998053,0.001947,0.000000
усы,0.333333,0.000000,0.666667


Альфа отражает осознанность говорящего. При росте этого показателя импликатруа получается "строже", потому что при дефолтном альфа = 1 распределение для очков было 0,6 на 0,4 для r1 и r2 соответственно, с увеличением альфа растет условная вероятность выбора r1 при высказывании очки и падает вероятность выбора r2, что усиливает импликатуру, что усы не очки и импликатуру, что очки не шляпа. 

In [41]:
for pr in pri:
    display_rsa(p=pr)

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.0,1.0,0.0,0.0
очки,1.0,1.0,0.0,0.0
усы,1.0,0.0,1.0,0.0
prior,0.3,0.4,0.3,
alpha,1.0,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.000000,1.000000,0.00
очки,0.487685,0.512315,0.00
усы,0.350000,0.000000,0.65


Even priors and all-0 message costs



...

,r1,r2,r3
шляпа,0.000000,1.000000,0.0
очки,0.205882,0.794118,0.0
усы,0.400000,0.000000,0.6


Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.0,1.0,0.0,0.0
очки,1.0,1.0,0.0,0.0
усы,1.0,0.0,1.0,0.0
prior,0.1,0.8,0.1,
alpha,1.0,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.00000,1.00000,0.00
очки,0.04607,0.95393,0.00
усы,0.45000,0.00000,0.55


При изменении априорных вероятностей есть тенденция, что тяжелее сделать вывод об импликатуре, так как при увеличении исходных вероятностей для конкурирующего признака (в данном случае очки), чем выше априорная, тем сильнее она будет влиять на распределение вероятностей и тем сложнее сказать, что очки – это не шляпа, а усы – это не очки. 

In [44]:
for co in cost:
    display_rsa(c=co)

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.000000,1.0,0.0,0
очки,1.000000,1.0,0.0,-7
усы,1.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.000000,1.000000,0.000000
очки,0.666565,0.333435,0.000000
усы,0.499772,0.000000,0.500228


Even priors and all-0 message costs



...

,r1,r2,r3
шляпа,0.000000,1.000000,0.000000
очки,0.652874,0.347126,0.000000
усы,0.468311,0.000000,0.531689


Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,0.000000,1.0,0.0,0
очки,1.000000,1.0,0.0,-1
усы,1.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.000000,1.000000,0.000000
очки,0.633842,0.366158,0.000000
усы,0.422319,0.000000,0.577681


С повышением цены высказывания повышается вероятность выбрать r1 при высказывании очки и понижается вероятность выбрать r2 при том же высказывании. Это можно проинтерпретировать так, что чем дороже/труднее высказывание для говорящего, тем больше он хочет, чтобы слушающий понял импликатуру, что очки не шляпа, когда говорящий все-таки решается произнести высказывание очки. То есть с повышеним цены высказывания можно сказать, что импликатура усиливается.  

# игра 2 

In [7]:
msgs = ['шляпа', 'очки', 'усы']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 1.0],
    [1.0, 1.0, 0.0]], index=msgs, columns=states)

print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod = RSA(lexicon=lex, prior=[float(1/3), float(1/3), float(1/3)], costs=[0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0.0
очки,0.000000,1.0,1.0,0.0
усы,1.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
шляпа,1.0,0.0,0.0
очки,0.0,0.5,0.5
усы,0.5,0.5,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,0.666667,0.0,0.333333
r2,0.000000,0.5,0.500000
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
шляпа,1.0,0.000000,0.000000
очки,0.0,0.333333,0.666667
усы,0.4,0.600000,0.000000


В игре 2 мы получили 2 скалярные импликатуры:
1) усы – не шляпа
2) очки – не усы

Игра 2 по типу похожа на игру 1, поэтому я решила анализ по показателям не дублировать и изобрать на первой задаче.

# Пункт 4

Наверное, можно рассмотреть применение RSA для фиксирования случаев омонимии (с какой вероятностью человек может случано подумать про другое значение омонимичного слова).

С помощью теории рационального речевого акта можно рассмотреть, как говорящий использует контекст и знания слушателя для выбора значения при таких индентичных словах. В контексте RSA цель говорящего состоит в том, чтобы передать сообщение слушателю с максимальной пользой и минимальными затратами на производство высказывания. Для достижения этой цели говорящий может выбирать значение слова на основе ожидаемых знаний и предположений слушателя.

На выбор значений, вероятно, как-то будут влиять исходные вероятности, которые будут обсуловлены контекстом и темой высказывания. Осознанность и рациональность (альфа) говорящего тоже может влиять на использование им слова, который является омонимом. 

Однако для того чтобы применять формулы RSA к лингвистическим феноменам, возможно, потребуются некоторые модификации, например, добавление параметров, учитывающих особенности языковой среды или некоторые дополнительные данные о слушателе.